# A股市场明细收集

## easyquotation使用示例

In [1]:
import easyquotation
import pandas as pd
import os

In [2]:
quotation = easyquotation.use('sina') # 新浪 ['sina'] 腾讯 ['tencent', 'qq'] 

In [3]:
#获取所有股票行情
# prefix 参数指定返回的行情字典中的股票代码 key 是否带 sz/sh 前缀
market_snapshot = quotation.market_snapshot(prefix=True) 

In [4]:
type(market_snapshot)

dict

In [5]:
market_df = pd.DataFrame(market_snapshot)

```{'sh000159': {'name': '国际实业', # 股票名
  'buy': 8.87, # 竞买价
  'sell': 8.88, # 竞卖价
  'now': 8.88, # 现价
  'open': 8.99, # 开盘价
  'close': 8.96, # 昨日收盘价
  'high': 9.15, # 今日最高价
  'low': 8.83, # 今日最低价
  'turnover': 22545048, # 交易股数
  'volume': 202704887.74， # 交易金额
  'ask1': 8.88, # 卖一价
  'ask1_volume': 111900, # 卖一量
  'ask2': 8.89,
  'ask2_volume': 54700,
  'bid1': 8.87, # 买一价
  'bid1_volume': 21800, # 买一量
  ...
  'bid2': 8.86, 
  'bid2_volume': 78400,
  'date': '2016-02-19',
  'time': '14:30:00',
  ...},
  ......```

In [6]:
market_df = market_df.T

In [7]:
market_df.head()

,name,open,close,now,high,low,buy,sell,turnover,volume,...,ask2_volume,ask2,ask3_volume,ask3,ask4_volume,ask4,ask5_volume,ask5,date,time
sz000001,平安银行,21.62,21.45,21.65,22.15,21.26,21.65,21.66,147342505,3.19654e+09,...,97400,21.67,37300,21.68,124698,21.69,52300,21.7,2021-03-02,15:00:03
sz000002,万科Ａ,33,33.35,33.29,34.6,32.9,33.28,33.29,122014989,4.11032e+09,...,16200,33.3,12000,33.31,29200,33.32,27100,33.33,2021-03-02,15:00:03
sz000003,PT金田A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-02-19,11:45:02
sz000004,国华网安,19.1,19.12,18.99,19.12,18.68,18.99,19,1872000,3.53105e+07,...,800,19.02,9100,19.05,3100,19.06,400,19.08,2021-03-02,15:00:03
sz000005,世纪星源,2.21,2.23,2.22,2.27,2.2,2.22,2.23,7964232,1.77529e+07,...,107300,2.24,279078,2.25,258800,2.26,267900,2.27,2021-03-02,15:00:03


In [8]:
def get_increase(arrLike, close, now):
    if arrLike[close] > 0:
        return round(((arrLike[now]-arrLike[close]) / arrLike[close]) * 100, 2)
    else:
        return 0

In [9]:
market_df['increase'] = market_df.apply(get_increase, axis=1, args=['close', 'now'])

In [10]:
market_df.head()

,name,open,close,now,high,low,buy,sell,turnover,volume,...,ask2,ask3_volume,ask3,ask4_volume,ask4,ask5_volume,ask5,date,time,increase
sz000001,平安银行,21.62,21.45,21.65,22.15,21.26,21.65,21.66,147342505,3.19654e+09,...,21.67,37300,21.68,124698,21.69,52300,21.7,2021-03-02,15:00:03,0.93
sz000002,万科Ａ,33,33.35,33.29,34.6,32.9,33.28,33.29,122014989,4.11032e+09,...,33.3,12000,33.31,29200,33.32,27100,33.33,2021-03-02,15:00:03,-0.18
sz000003,PT金田A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2021-02-19,11:45:02,0.00
sz000004,国华网安,19.1,19.12,18.99,19.12,18.68,18.99,19,1872000,3.53105e+07,...,19.02,9100,19.05,3100,19.06,400,19.08,2021-03-02,15:00:03,-0.68
sz000005,世纪星源,2.21,2.23,2.22,2.27,2.2,2.22,2.23,7964232,1.77529e+07,...,2.24,279078,2.25,258800,2.26,267900,2.27,2021-03-02,15:00:03,-0.45


In [12]:
market_df[(market_df['increase'] != -100) & (market_df['date'] == '2021-03-02')]['increase'].describe()

count    5028.000000
mean       -0.579982
std         2.280039
min       -11.730000
25%        -1.630000
50%        -0.760000
75%         0.140000
max        20.000000
Name: increase, dtype: float64

In [13]:
market_df['now'].describe()

count     5419.0
unique    3321.0
top          0.0
freq       175.0
Name: now, dtype: float64

In [14]:
stocks_folder = './excel/stocks/'
if not os.path.exists(stocks_folder):
    os.makedirs(stocks_folder)

In [15]:
import time
try:
    excel_name = 'China_Stocks_Trade_{0}.xlsx'.format(
        time.strftime('%Y%m%d%H%M%S', time.localtime(time.time())))
    stocks_folder = './excel/stocks/'
    if not os.path.exists(stocks_folder):
        os.makedirs(stocks_folder)
    excel_filepath = os.path.join(stocks_folder,
                                  excel_name)
    write = pd.ExcelWriter(excel_filepath)
    market_df.to_excel(write,
                       sheet_name='details',
                       index=True,
                       encoding='utf-8')
    write.save()
except Exception as e:
    print(e)

## tushare使用示例

In [10]:
import tushare as ts
import os
import pandas as pd
import time

In [22]:
profit_data = ts.get_profit_data(2020,4)

本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
[Getting data:]##############

In [23]:
profit_data.head()

,code,name,roe,net_profit_ratio,gross_profit_rate,net_profits,eps,business_income,bips
0,300951,博硕科技,43.37,23.88,47.9108,163.9801,2.7330,686.4022,11.440
1,688628,优利德,36.14,NaN,NaN,150.6118,1.8255,NaN,NaN
2,688677,海泰新光,29.75,NaN,NaN,93.0980,1.4278,NaN,NaN
3,300949,奥雅设计,25.11,20.87,NaN,102.8659,NaN,492.6927,NaN
4,002107,沃华医药,22.15,17.77,77.8340,178.8804,0.3099,1006.0815,1.743


```
code,代码
name,名称
roe,净资产收益率(%)
net_profit_ratio,净利率(%)
gross_profit_rate,毛利率(%)
net_profits,净利润(万元)
esp,每股收益
business_income,营业收入(百万元)
bips,每股主营业务收入(元)
```

In [24]:
import time
try:
    excel_name = 'China_Stocks_Profit_2020_4_{0}.xlsx'.format(
        time.strftime('%Y%m%d%H%M%S', time.localtime(time.time())))
    stocks_folder = './excel/stocks/'
    if not os.path.exists(stocks_folder):
        os.makedirs(stocks_folder)
    excel_filepath = os.path.join(stocks_folder,
                                  excel_name)
    write = pd.ExcelWriter(excel_filepath)
    profit_data.to_excel(write,
                        sheet_name='details',
                        index=True,
                        encoding='utf-8')
    write.save()
except Exception as e:
    print(e)

### 获取当前所有正常上市交易的股票列表

In [3]:
pro = ts.pro_api('31540a474cc888b4c8c280ed0621a147b718e78ee485f82c0c6a1d90')

In [4]:
data = pro.stock_basic(exchange='', list_status='L')

In [5]:
data.head()

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,国华网安,深圳,互联网,主板,19910114
3,000005.SZ,000005,世纪星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427


In [13]:
stocks_folder = './excel/stocks/tushare/'
excel_name = '中国A股股票列表_{0}.xlsx'.format(time.strftime('%Y%m%d%H%M%S', time.localtime(time.time())))

In [14]:
def export_excel(folder: str, file_name: str, data_dict: dict):
    import time
    try:
        if not os.path.exists(folder):
            os.makedirs(folder)
        excel_filepath = os.path.join(folder,
                                      file_name)
        write = pd.ExcelWriter(excel_filepath)
        for sheet_name, data_df in data_dict.items():
            data_df.to_excel(write,
                             sheet_name=sheet_name,
                             index=True,
                             encoding='utf-8')
        write.save()
    except Exception as e:
        print(e)

In [15]:
export_excel(stocks_folder, excel_name, {'股票列表': data})

In [22]:
pro.hsgt_top10(trade_date='20210226', market_type='3')

,trade_date,ts_code,name,close,change,rank,market_type,amount,net_amount,buy,sell
0,20210226,000002.SZ,万科A,33.10,0.3334,4,3,2.150206e+09,152628515.0,1.151418e+09,9.987890e+08
1,20210226,000333.SZ,美的集团,93.08,-0.9155,1,3,4.175229e+09,322832267.0,2.249031e+09,1.926198e+09
2,20210226,000568.SZ,泸州老窖,235.21,-3.2734,9,3,1.242879e+09,120826120.0,6.818526e+08,5.610265e+08
3,20210226,000858.SZ,五粮液,280.00,-3.0807,2,3,3.294506e+09,-873176900.0,1.210665e+09,2.083842e+09
4,20210226,002460.SZ,赣锋锂业,99.02,-8.1447,10,3,1.194644e+09,265136832.0,7.298903e+08,4.647534e+08
5,20210226,002594.SZ,比亚迪,197.27,-5.7252,8,3,1.366454e+09,-2343044.0,6.820553e+08,6.843983e+08
6,20210226,002714.SZ,牧原股份,113.90,-8.6535,6,3,1.682154e+09,301057919.0,9.916062e+08,6.905483e+08
7,20210226,300059.SZ,东方财富,30.38,-4.0733,7,3,1.478604e+09,10816801.0,7.447103e+08,7.338935e+08
8,20210226,300274.SZ,阳光电源,86.48,-0.9620,5,3,1.951939e+09,460380707.0,1.206160e+09,7.457790e+08
9,20210226,300750.SZ,宁德时代,322.23,-4.4112,3,3,2.550062e+09,303164960.0,1.426613e+09,1.123448e+09


In [23]:
df = pro.daily(trade_date='20210226')

In [24]:
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20210226,21.60,21.96,21.21,21.38,22.09,-0.71,-3.2141,1124977.30,2424653.753
1,000002.SZ,20210226,32.57,34.08,31.62,33.10,32.99,0.11,0.3334,2529180.39,8329307.819
2,000004.SZ,20210226,18.75,19.16,18.53,18.85,18.81,0.04,0.2127,20284.00,38245.024
3,000005.SZ,20210226,2.11,2.15,2.09,2.14,2.13,0.01,0.4695,60606.95,12889.087
4,000006.SZ,20210226,5.29,5.30,5.16,5.16,5.39,-0.23,-4.2672,164241.70,85582.107


In [25]:
df = pro.fina_indicator(ts_code='600000.SH')

Exception: 抱歉，您没有访问该接口的权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。